### --- Day 18: Lavaduct Lagoon ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/18

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines =  LoadPuzzleInput(2023, 18);
WriteLines(inputLines);

Loading puzzle file: Day18.txt
Total lines: 624
Max line length: 14

R 4 (#5ca782)
U 6 (#79d133)
R 8 (#90d7f2)
U 3 (#13cc11)
R 3 (#57f4c0)


In [4]:
record Point(int row, int col) {
    public static Point operator +(Point a, Point b) => new(a.row + b.row, a.col + b.col);

    public static Point operator *(Point a, int x) => new(a.row * x, a.col * x);
}

Point Down = new(1, 0);
Point Up = new(-1, 0);
Point Right = new(0, 1);
Point Left = new(0, -1);

Point GetDir(char c) => c switch {
    'U' => Up,
    'D' => Down,
    'L' => Left,
    'R' => Right,
    _ => throw new Exception("Unexpected char")
};

record Step(char direction, int steps, string colour);

record TracePoint(Point point, bool isWall, Step source);

IEnumerable<(T item, int i)> Enumerate<T>(IEnumerable<T> source) => source.Select((item, i) => (item, i));

IList<TracePoint> TraceAllPoints(IList<Step> steps) {
    Dictionary<Point, TracePoint> result = new();
    
    Point p = new(0, 0);

    var prev = steps.Last();
    foreach (var (step, i) in Enumerate(steps))
    {
        var next = steps[(i + 1) % steps.Count];

        var directions = (prev.direction, step.direction, next.direction);
        var isWall = directions switch {
            (_, 'D', _) => true,
            (_, 'U', _) => true,
            ('U', _, 'U') => true,
            ('D', _, 'D') => true,
            _ => false
        };

        if ("LR".Contains(step.direction)) {
            // Assign the last of the pervious vertical step it to the current
            // horizontal step, so we can "group" the wall pieces
            
            result[p] = new(p, isWall, step);
        }

        foreach (var st in Enumerable.Range(0, step.steps)) {
            p += GetDir(step.direction);
            
            if (!result.ContainsKey(p)) {
                // Can happen for 0,0
                result[p] = new(p, isWall, step);
            }
        }

        prev = step;
    }

    return result.Values.ToList();
}

In [5]:
Console.WriteLine(string.Join('\n', TraceAllPoints([new('R', 3, "red"), new('U', 3, "blue")])));

TracePoint { point = Point { row = 0, col = 0 }, isWall = True, source = Step { direction = R, steps = 3, colour = red } }
TracePoint { point = Point { row = 0, col = 1 }, isWall = True, source = Step { direction = R, steps = 3, colour = red } }
TracePoint { point = Point { row = 0, col = 2 }, isWall = True, source = Step { direction = R, steps = 3, colour = red } }
TracePoint { point = Point { row = 0, col = 3 }, isWall = True, source = Step { direction = R, steps = 3, colour = red } }
TracePoint { point = Point { row = -1, col = 3 }, isWall = True, source = Step { direction = U, steps = 3, colour = blue } }
TracePoint { point = Point { row = -2, col = 3 }, isWall = True, source = Step { direction = U, steps = 3, colour = blue } }
TracePoint { point = Point { row = -3, col = 3 }, isWall = True, source = Step { direction = U, steps = 3, colour = blue } }


In [6]:
Step ParseStep(string line) {
    var bits = line.Split(' ');

    var direction = bits[0][0];
    var steps = int.Parse(bits[1]);
    var colour = bits[2].Substring(2, 6);

    return new(direction, steps, colour);
}

In [7]:
var testInput = """
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
""";
var testInputLines = testInput.Split('\n');
var testInputSteps = testInputLines.Select(ParseStep).ToList();

var testInputTrace = TraceAllPoints(testInputSteps).ToList();
Console.WriteLine(testInputTrace.Count);
Console.WriteLine(string.Join('\n', testInputTrace.Take(5)));

38
TracePoint { point = Point { row = 0, col = 0 }, isWall = False, source = Step { direction = R, steps = 6, colour = 70c710 } }
TracePoint { point = Point { row = 0, col = 1 }, isWall = False, source = Step { direction = R, steps = 6, colour = 70c710 } }
TracePoint { point = Point { row = 0, col = 2 }, isWall = False, source = Step { direction = R, steps = 6, colour = 70c710 } }
TracePoint { point = Point { row = 0, col = 3 }, isWall = False, source = Step { direction = R, steps = 6, colour = 70c710 } }
TracePoint { point = Point { row = 0, col = 4 }, isWall = False, source = Step { direction = R, steps = 6, colour = 70c710 } }


In [8]:
IEnumerable<Point> GetInternals(IEnumerable<TracePoint> tracePoints) {
    HashSet<Point> existing = new(tracePoints.Select(tp => tp.point));

    // For a given row, the current column is inside the shape if it's within an odd-numbered number of walls

    var q = from tp in tracePoints
            where tp.isWall
            group tp by tp.point.row into tpRow
            let tpg = tpRow.GroupBy(tpr => tpr.source).OrderBy(g => g.Min(gg => gg.point.col))
            let tpgPairs = Enumerate(tpg).GroupBy(ti => ti.i / 2).Where(g => g.Count() == 2)
            let tpgPairTuples = tpgPairs.Select(p => p.ToArray()).Select(p => (a: p[0].item, b: p[1].item))
            from tp in tpgPairTuples
            let lastA = tp.a.Max(a => a.point.col)
            let firstB = tp.b.Min(b => b.point.col)
            let start = lastA + 1
            let cols = firstB - start
            // Make sure we don't double-count horizontal pieces that are not
            // counted as "walls"
            from i in Enumerable.Range(0, cols)
            let p = new Point(tpRow.Key, start) + (Right * i)
            where !existing.Contains(p)
            select p;

    return q;
}

foreach (var x in GetInternals(testInputTrace).Take(10)) {
    Console.WriteLine(x);
}

Point { row = 1, col = 1 }
Point { row = 1, col = 2 }
Point { row = 1, col = 3 }
Point { row = 1, col = 4 }
Point { row = 1, col = 5 }
Point { row = 2, col = 3 }
Point { row = 2, col = 4 }
Point { row = 2, col = 5 }
Point { row = 3, col = 3 }
Point { row = 3, col = 4 }


In [9]:
void Render(IEnumerable<TracePoint> points) {
    var minRow = points.Min(p => p.point.row);
    var minCol = points.Min(p => p.point.col);

    var correction = new Point(-minRow, -minCol);

    var pointDict = points.ToDictionary(tp => tp.point + correction);

    var rows = pointDict.Keys.Max(p => p.row);
    var cols = pointDict.Keys.Max(p => p.col);

    var chars = new char[rows + 1][];

    for (var row = 0; row <= rows; row++) {
        chars[row] = new char[cols + 1];
        for (var col = 0; col <= cols; col++) {
            var p = new Point(row, col);

            char ch = '.';

            chars[row][col] = ch;
        }
    }

    foreach (var p in points) {
        var (row, col) = p.point + correction;
        chars[row][col] = p.isWall ? '|' : '-';
    }

    var internals = GetInternals(points).Select(p => p + correction);
    foreach (var (row, col) in internals) {
        chars[row][col] = 'x';
    }

    Console.WriteLine(string.Join('\n', chars.Select(row => new string(row))));
}

Render(testInputTrace);

-------
|xxxxx|
|||xxx|
..|xxx|
..|xxx|
|||x|||
|xxx|..
||xx|||
.|xxxx|
.------


In [10]:
// Now, the lagoon can contain a much more respectable 62 cubic meters of lava. 

var testTotal = testInputSteps.Sum(st => st.steps) + GetInternals(testInputTrace).Count();
Console.WriteLine(testTotal);

62


In [11]:
var inputSteps = inputLines.Select(ParseStep).ToList();
var inputTrace = TraceAllPoints(inputSteps);

// Big ascii art here
// Render(inputTrace);

In [12]:
// The Elves are concerned the lagoon won't be large enough; if they follow
// their dig plan, how many cubic meters of lava could it hold?

var part1Answer = inputSteps.Sum(x => x.steps) + GetInternals(inputTrace).Count();
Console.WriteLine(part1Answer);

47139


In [13]:
// 47139 is correct!
Ensure(47139, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/18

In [15]:
int Number(string s) => int.Parse(s, System.Globalization.NumberStyles.HexNumber);

char[] directionIndex = ['R', 'D', 'L', 'U'];

Step ParseStep2(string line) {
    var bits = line.Split(' ');

    var stepsHex = bits[2].Substring(2, 5);
    var steps = Number(stepsHex);
    var dirInt = int.Parse(bits[2].Substring(7,1));
    var dirChar = directionIndex[dirInt];

    return new Step(dirChar, steps, stepsHex);
}

var testInputSteps2 = testInputLines.Select(ParseStep2);
foreach (var t in testInputSteps2) {
    Console.WriteLine(t);
}

Step { direction = R, steps = 461937, colour = 70c71 }
Step { direction = D, steps = 56407, colour = 0dc57 }
Step { direction = R, steps = 356671, colour = 5713f }
Step { direction = D, steps = 863240, colour = d2c08 }
Step { direction = R, steps = 367720, colour = 59c68 }
Step { direction = D, steps = 266681, colour = 411b9 }
Step { direction = L, steps = 577262, colour = 8ceee }
Step { direction = U, steps = 829975, colour = caa17 }
Step { direction = L, steps = 112010, colour = 1b58a }
Step { direction = D, steps = 829975, colour = caa17 }
Step { direction = L, steps = 491645, colour = 7807d }
Step { direction = U, steps = 686074, colour = a77fa }
Step { direction = L, steps = 5411, colour = 01523 }
Step { direction = U, steps = 500254, colour = 7a21e }


In [16]:
// See if we can reduce these to blocks
static ulong GCD(ulong a, ulong b) {
    if (b == 0) {
        return a;
    }

    return GCD(b, a % b);
}

var horizontals = testInputSteps2.Where(s => "LR".Contains(s.direction)).Select(s => s.steps).ToList();
foreach (var h in horizontals) {
    Console.WriteLine(h);
}
var horizontalGCD = horizontals.Take(3).Select(h => (ulong)h).Aggregate(GCD);
Console.WriteLine(horizontalGCD);

461937
356671
367720
577262
112010
491645
5411
1


In [17]:
// Ok, so we can't scale :D. Let's try maintaing our lines as actual lines and breaking up the grid into sectors

record TraceLine(Point point, char direction, int extraSteps, bool isWall, Step source);

IList<TraceLine> TraceAllLines(IList<Step> steps) {
    Dictionary<Step, TraceLine> result = new();
    
    Point p = new(0, 0);

    var prev = steps[steps.Count - 1];
    foreach (var (step, i) in Enumerate(steps))
    {
        var next = steps[(i + 1) % steps.Count];

        var directions = (prev.direction, step.direction, next.direction);
        var isWall = directions switch {
            (_, 'D', _) => true,
            (_, 'U', _) => true,
            ('U', _, 'U') => true,
            ('D', _, 'D') => true,
            _ => false
        };

        var traceDir = GetDir(step.direction);
        var traceStart = p + traceDir;
        var traceExtraSteps = step.steps - 1;
        var newLine = new TraceLine(traceStart, step.direction, traceExtraSteps, isWall, step);

        if ("LR".Contains(step.direction)) {
            // The previous line terminates on our line. Move our start position
            // across 1 block, to consume their vertical block as our horizontal
            // one
            
            newLine = newLine with { 
                point = newLine.point + traceDir * -1,
                extraSteps = newLine.extraSteps + 1
             };

            // Likewise, shorten the length of the previous vertical block by
            // one

            var prevDir = GetDir(prev.direction);
            var prevStart = p + (prevDir * -prev.steps) + prevDir;
            var newPrev = new TraceLine(prevStart, prev.direction, prev.steps - 2, true, prev);
            result[prev] = newPrev;
        }

        p += GetDir(step.direction) * step.steps;

        if (!result.ContainsKey(step)) {
            // Can happen for 0,0
            result[step] = newLine;
        }

        prev = step;
    }

    return result.Values.ToList();
}

Step[] testSquare = [
    new ('R', 5, "x"),
    new ('D', 5, "x"),
    new ('L', 5, "x"),
    new ('U', 5, "x")
];
var testSquareTraceLines = TraceAllLines(testSquare);

foreach (var st in testSquareTraceLines) {
    Console.WriteLine(st);
}

TraceLine { point = Point { row = 4, col = 0 }, direction = U, extraSteps = 3, isWall = True, source = Step { direction = U, steps = 5, colour = x } }
TraceLine { point = Point { row = 0, col = 0 }, direction = R, extraSteps = 5, isWall = False, source = Step { direction = R, steps = 5, colour = x } }
TraceLine { point = Point { row = 1, col = 5 }, direction = D, extraSteps = 3, isWall = True, source = Step { direction = D, steps = 5, colour = x } }
TraceLine { point = Point { row = 5, col = 5 }, direction = L, extraSteps = 5, isWall = False, source = Step { direction = L, steps = 5, colour = x } }


In [18]:
// Normalise to Right or Down lines, so the end point is always larger than the
// start

IEnumerable<TraceLine> Normalise(IEnumerable<TraceLine> traceLines) {
    foreach (var tl in traceLines) {
        var (startRow, startCol) = tl.point;
        var (endRow, endCol) = tl.point + GetDir(tl.direction) * (tl.extraSteps);

        yield return tl.direction switch {
            'L' => new(new(endRow, endCol), 'R', tl.extraSteps, tl.isWall, tl.source),
            'U' => new(new(endRow, endCol), 'D', tl.extraSteps, tl.isWall, tl.source),
            _ => tl
        };
    }
}

In [19]:
IEnumerable<int> RowJunctions(IEnumerable<TraceLine> traceLines) {
    var rowStarts = traceLines.Select(tl => tl.point.row);

    // Handle the case where a vertical line starts on the same row as a
    // horizontal line. In this case the horizontal line height is 1 while the
    // vertical line may be greater than 1. Handle this by adding the following row
    // as a junction point also, ensuring that all vertical lines that share a row
    // with a horizontal line will also have a height 0
    var afterLines = traceLines.Where(tl => tl.direction == 'R').Select(tl => tl.point.row + 1);

    var rowJunctions = rowStarts.Concat(afterLines)
        .Distinct()
        .OrderBy(row => row)
        .ToList();

    return rowJunctions;
}

In [20]:
IEnumerable<TraceLine> ChopVerticals(IEnumerable<TraceLine> traceLines) {
    var verticalJunctions = RowJunctions(traceLines.ToList());

    foreach (var tl in traceLines) {
        if (tl.direction == 'R') {
            // Horizontal. No chop
            yield return tl;
            continue;
        }

        // Vertical line. Chop at all the junction points between the start and end

        var start = tl.point;
        var startRow = tl.point.row;
        var endRow = startRow + tl.extraSteps;

        var bits = verticalJunctions.Where(vj => vj > startRow && vj <= endRow).Append(startRow).Append(endRow + 1).OrderBy(vj => vj);
        var splits = bits.Zip(bits.Skip(1));

        foreach (var s in splits) {
            var (a, b) = s;

            var extraSteps = b - a - 1;

            Point startPoint = new(a, tl.point.col);
            var newLine = new TraceLine(startPoint, tl.direction, extraSteps, tl.isWall, tl.source);
            yield return newLine;
        }
    }
}

In [21]:
// At this point all the vertical lines have been chopped at all significant
// horizontal points, and all lines on the same row have the same height. Therefore
// we should be able to perform our horizontal scan again

record Sector(Point point, int rows, int cols);

int RowHeight(TraceLine tl) => tl.direction switch {
    'D' => 1 + tl.extraSteps,
    'R' => 1,
    _ => throw new Exception("Unexpected direction")
};

IEnumerable<Sector> GetInternals2(IEnumerable<TraceLine> traceLines) {
    HashSet<Point> existing = new(traceLines.Select(tp => tp.point));

    // For a given row, the current column is inside the shape if it's within an odd-numbered number of walls

    var traceLineRows = from tl in traceLines
                        group tl by tl.point.row into tlRow
                        select tlRow.OrderBy(tlr => tlr.point.col).ToList();

    foreach (var row in traceLineRows) {
        var rowHeight = RowHeight(row.First()); // all lines in a row have same height

        Point scanPoint = null;
        foreach (var line in row) {
            if (scanPoint != null) {
                // We've hit another line inside the shape
                var cols = line.point.col - scanPoint.col;
                var rows = rowHeight;
                if (line.isWall) {
                    // We've run into the next wall. Return the internal sector and now we are outside
                    yield return new Sector(scanPoint, rows, cols);
                    scanPoint = null;
                } else {
                    // We've run into a non-wall horizontal piece. Return the current sector but stay within the wall; advance the pointer
                    yield return new Sector(scanPoint, rows, cols);
                    var newCol = line.point.col + 1 + line.extraSteps;
                    scanPoint = new(line.point.row, newCol);
                }
            } else {
                if (line.isWall) {
                    // We've hit an outer wall. Track the pointer as we are now inside the shape
                    var advanceCols = 1 + (line.direction == 'D' ? 0 : line.extraSteps);
                    scanPoint = new(line.point.row, line.point.col + advanceCols);
                }
            }
        }
    }
}

var testInputTraceLines = TraceAllLines(testInputSteps);
var testInputTraceInternals = GetInternals2(ChopVerticals(Normalise(testInputTraceLines)));
var testInputTraceLinesTotal = testInputTraceInternals.Select(i => i.rows * i.cols).Sum() 
                                + testInputTraceLines.Select(tl => 1 + tl.extraSteps).Sum();

// Match the original 62 from the Pt 1 sample
Console.WriteLine(testInputTraceLinesTotal);

62


In [22]:
// Re-test Pt 1 using this sector scanning
var inputTraceLines = TraceAllLines(inputSteps);
var inputTraceLinesChopped = ChopVerticals(Normalise(inputTraceLines)).ToList();
var inputTraceLinesInternal = GetInternals2(inputTraceLinesChopped).ToList();
var inputTraceLinesTotal = inputTraceLinesInternal.Select(i => i.rows * i.cols).Sum()
                            + inputTraceLinesChopped.Sum(tl => 1 + tl.extraSteps);

// Hoping to match the 47139
Console.WriteLine(inputTraceLinesTotal);

47139


In [23]:
var inputSteps2 = inputLines.Select(ParseStep2).ToList();
var inputTraceLines2 = TraceAllLines(inputSteps2);
var inputTraceLines2Chopped = ChopVerticals(Normalise(inputTraceLines2)).ToList();
var inputTraceLines2Internal = GetInternals2(inputTraceLines2Chopped).ToList();

var part2Answer = inputTraceLines2Internal.Select(i => (long)i.rows * (long)i.cols).Sum() 
                                + inputTraceLines2Chopped.Sum(tl => 1 + (long)tl.extraSteps);

// Convert the hexadecimal color codes into the correct instructions; if the
// Elves follow this new dig plan, how many cubic meters of lava could the lagoon
// hold?
Console.WriteLine(part2Answer);

173152345887206


In [24]:
// 173152345887206 is correct!
Ensure(173152345887206, part2Answer);

In [25]:
// Consider revisiting this one. Consider the shoelace algorithm and Pikes(??) algorithm?